In [20]:
from firecrawl import FirecrawlApp, ScrapeOptions

In [31]:
app = FirecrawlApp(api_key="fc-d2043bb0efb94369838b71c7bcb072ac")

In [22]:
def crawl_website(root_url, limit=10):
    print(f"Starting crawl of {root_url}")
    result = app.crawl_url(root_url, limit=limit, scrape_options=ScrapeOptions(formats=['markdown']))
    return result.data

In [11]:
result=crawl_website('https://www.kapture.cx/')

Starting crawl of https://www.kapture.cx/


In [26]:
result[:2]

[FirecrawlDocument(url=None, markdown='# Kapture CX Recognized in FT High-Growth Companies Asia-Pacific 2025\n\nIndia, 18th March – Kapture, a leading [customer experience platform](https://www.kapture.cx/blog/customer-experience-platform/),, is proud to announce its inclusion in the prestigious Financial Times list of High-Growth Companies Asia-Pacific 2025. Kapture has secured a commendable ranking, recognizing its significant revenue growth and impact in the region.\n\nThe Financial Times, in collaboration with Statista, compiles this annual ranking to identify and celebrate the top 500 companies in the Asia-Pacific region that have demonstrated exceptional revenue growth between 2020 and 2023, and a CAGR of 44.64%.. This recognition underscores Kapture’s commitment to innovation, customer satisfaction, and strategic expansion.\n\n“We are thrilled to be recognized by the Financial Times as one of the fastest-growing companies in the Asia-Pacific region,” said Sheshgiri Kamath, Co-fo

In [24]:
for ele in result:
    print(ele)

url=None markdown='# Kapture CX Recognized in FT High-Growth Companies Asia-Pacific 2025\n\nIndia, 18th March – Kapture, a leading [customer experience platform](https://www.kapture.cx/blog/customer-experience-platform/),, is proud to announce its inclusion in the prestigious Financial Times list of High-Growth Companies Asia-Pacific 2025. Kapture has secured a commendable ranking, recognizing its significant revenue growth and impact in the region.\n\nThe Financial Times, in collaboration with Statista, compiles this annual ranking to identify and celebrate the top 500 companies in the Asia-Pacific region that have demonstrated exceptional revenue growth between 2020 and 2023, and a CAGR of 44.64%.. This recognition underscores Kapture’s commitment to innovation, customer satisfaction, and strategic expansion.\n\n“We are thrilled to be recognized by the Financial Times as one of the fastest-growing companies in the Asia-Pacific region,” said Sheshgiri Kamath, Co-founder & CEO at Kaptu

In [23]:
import json
from typing import List, Dict
import markdown as md
from bs4 import BeautifulSoup
import re

def extract_pages_to_json(
    docs: List[object]) -> json:
    """
    From a list of FirecrawlDocument-like objects, writes a JSON list where each entry has:
      - text: cleaned, tag-free, link-free
      - file_links: [.pdf/.docx/etc URLs]
      - image_links: [all image URLs from <img>]
    """
    # Define which extensions count as “files”
    doc_exts = ('.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx')
    url_strip = re.compile(r'https?://\S+')

    output: List[Dict[str, object]] = []

    for doc in docs:
        # 1) Get HTML source
        if getattr(doc, "rawHtml", None):
            html = doc.rawHtml
        elif getattr(doc, "html", None):
            html = doc.html
        else:
            md_src = getattr(doc, "markdown", "") or ""
            html = md.markdown(md_src)

        soup = BeautifulSoup(html, "html.parser")

        # 2) Find and classify all <a href> links
        file_links = []
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if href.lower().endswith(doc_exts):
                file_links.append(href)

        # 3) Find all <img> src URLs
        image_links = [img["src"] for img in soup.find_all("img", src=True)]

        # 4) Extract clean text
        text = soup.get_text(separator="\n")
        text = url_strip.sub("", text)           # remove any leftover URLs
        text = re.sub(r"\n{3,}", "\n\n", text).strip()

        output.append({
            "text": text,
            "file_links": file_links,
            "image_links": image_links
        })

    # # 5) Write JSON
    # with open(output_path, "w", encoding="utf-8") as f:
    #     json.dump(output, f, ensure_ascii=False, indent=2)
    return output


In [43]:
data=extract_pages_to_json(result, "kapture_pages.json")

In [44]:
text=""
for e in data:
    text+=e['text']

In [45]:
print(text)

Kapture CX Recognized in FT High-Growth Companies Asia-Pacific 2025

India, 18th March – Kapture, a leading 
customer experience platform
,, is proud to announce its inclusion in the prestigious Financial Times list of High-Growth Companies Asia-Pacific 2025. Kapture has secured a commendable ranking, recognizing its significant revenue growth and impact in the region.

The Financial Times, in collaboration with Statista, compiles this annual ranking to identify and celebrate the top 500 companies in the Asia-Pacific region that have demonstrated exceptional revenue growth between 2020 and 2023, and a CAGR of 44.64%.. This recognition underscores Kapture’s commitment to innovation, customer satisfaction, and strategic expansion.

“We are thrilled to be recognized by the Financial Times as one of the fastest-growing companies in the Asia-Pacific region,” said Sheshgiri Kamath, Co-founder & CEO at Kapture. “This achievement is a testament to the hard work and dedication of our team, as w

In [19]:
import time

In [6]:
st=time.time()
sec=crawl_website('https://support.sendcloud.com/hc/en-us',limit=50)
print("Time_Taken to scrap the data=",time.time()-st)

Starting crawl of https://support.sendcloud.com/hc/en-us
Time_Taken to scrap the data= 242.8481523990631


In [54]:
data=extract_pages_to_json(sec)

In [12]:
def extract_text(extracted:json) -> str:
    text=""
    for ele in extracted:
        text+=ele['text']
    return text

In [13]:
def unique_pdf(extracted:json) -> List[str]:
    links=[]
    for ele in extracted:
        if len(ele['file_links'])>0:
            for el in ele['file_links']:
                if el not in links:
                    links.append(el)
    return links

In [15]:
text=extract_text(data)
print(text)

Skip to main content

Sendcloud Help Center

Sendcloud Help Center

Peak Season FAQ

What peak surcharges can I expect this Peak Season?

Will the carriers follow different operating schedules during the holidays?

Are there any carrier disruptions due to peak season?

My parcel is delayed, what do I do?

How can I request a pickup?

How can I prevent surcharges?

Getting started

Features

Carriers

Integrations

Billing

FAQ

Not a Sendcloud user yet?

Start your 14-day trial

API Documentation
 
Sendcloud Academy

iframe
iframeSkip to main content

Return to top

Purpose
: learn how to connect Mijnwebwinkel to your Sendcloud account by following four easy steps.

Step 1:

For connecting to Mijnwebwinkel with Sendcloud you need a Professional Mijnwebwinkel account.

Go to 'Settings' and click on 'API'.

Step 2:

Click on the button 
Token toevoegen
.

2. The following screen will appear:

3. When adding a token you will see a form as shown in the next image. Choose at 'Partner' for S

In [34]:
links=unique_pdf(data)
print(links)

['https://www.postnl.nl/api/assets/blt43aa441bfc1e29f2/blta8f03d024c5258d6/678e16b218352e427d810b22/aanpassingenproduct-en-aanlevervoorwaarden-buitenland-en-2025.pdf', 'https://www.sendcloud.com/wp-content/help-center-images/DPD%20NL/Machtingsformulier%20EN%20%284%29.pdf', 'https://www.sendcloud.fr/wp-content/uploads/2018/04/ISO-Country-Codes.xlsx', 'https://www.dpd.com/wp-content/uploads/sites/220/2023/11/DPD2311_Toeslagen_NL_2024_2.pdf', 'https://www.dpd.com/dpd-nl/wp-content/uploads/sites/220/2019/10/PCoverzicht_EU_Uitzonderingsgebieden_EN_0719.pdf']


In [24]:
import io
import requests
import pdfplumber
from typing import List, Dict

def download_and_extract_pdfs(
    urls: List[str]
) -> Dict[str, str]:
    extracted: Dict[str, str] = {}
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/pdf",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "identity",  # Avoid compression issues
        "Referer": "https://google.com"
    }
    for url in urls:
        try:
            print(url)
            # 1) download into memory
            resp = requests.get(url, headers=headers, timeout=(10,60))
            resp.raise_for_status()

            # 2) open with pdfplumber from a BytesIO buffer
            with pdfplumber.open(io.BytesIO(resp.content)) as pdf:
                # join page texts with double newlines
                text = "\n\n".join(
                    page.extract_text() or "" for page in pdf.pages
                ).strip()

            extracted[url] = text

        except Exception as e:
            print(f"⚠️ Failed to process {url}: {e}")
            extracted[url] = ""

    return extracted


In [25]:
def download_extract_text_from_pdf(links:str):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept": "application/pdf",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "identity",  # Avoid compression issues
        "Referer": "https://google.com"
    }
    all_content=""
    for link in links:
        try:
            resp = requests.get(link, headers=headers, timeout=(10,60))
            resp.raise_for_status()
        except Exception as e:
            print(f"{link}={e}")
        # 2) open with pdfplumber from a BytesIO buffer
        if ".pdf" in link:
            with pdfplumber.open(io.BytesIO(resp.content)) as pdf:
                # join page texts with double newlines
                text = "\n\n".join(
                    page.extract_text() or "" for page in pdf.pages
                ).strip()
        else:
            continue
        all_content+=link+"\n"+text+"\n"
    return all_content

In [26]:
def unique_pdf(links:List[str],available:List[str])->List[str]:
    checked=[]
    for link in links:
        if link not in available:
            checked.append(link)
    return checked

In [27]:
def all_content_formatting(extracted:json) -> str:
    '''
        It will accept the extracted information from wesite and format everythink
    '''
    all_text:str=""
    used_pdf=[]
    for field in extracted:
        all_text+=field['text']
        new_links=unique_pdf(field['file_links'],used_pdf)
        used_pdf.extend(new_links)
        content_from_pdf=download_extract_text_from_pdf(new_links)
        all_text+="\n"+content_from_pdf+"\n"
    return all_text


In [32]:
st=time.time()
sec=crawl_website('https://support.sendcloud.com/hc/en-us',limit=50)
print("Time_Taken to scrap the data=",time.time()-st)

Starting crawl of https://support.sendcloud.com/hc/en-us
Time_Taken to scrap the data= 254.31870770454407


In [34]:
data=extract_pages_to_json(sec)
data_=all_content_formatting(data)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [35]:
print(data_)

Skip to main content

Sendcloud Help Center

Sendcloud Help Center

Peak Season FAQ

What peak surcharges can I expect this Peak Season?

Will the carriers follow different operating schedules during the holidays?

Are there any carrier disruptions due to peak season?

My parcel is delayed, what do I do?

How can I request a pickup?

How can I prevent surcharges?

Getting started

Features

Carriers

Integrations

Billing

FAQ

Not a Sendcloud user yet?

Start your 14-day trial

API Documentation
 
Sendcloud Academy

iframe
iframe

Skip to main content

Return to top

Purpose:
 Learn how to get support from Sendcloud when you have an issue or a question about your account, your shipments or your integration.

Opening hours:

Our support team is available Monday-Friday from 9:00-17:30 GMT+1.

UK support is available from 8.30-17.00 GMT.

Multi-channel support

To help you have the best possible experience when using Sendcloud, we offer multi-channel support so you can get help with your

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,   
    chunk_overlap=72,
    length_function=len,
)
chunked_text = text_splitter.split_text(data_)

In [178]:
from pymilvus import MilvusClient,FieldSchema,DataType,CollectionSchema, Collection, connections, utility
milvus_client = MilvusClient(uri="http://127.0.0.1:19530",token="root:Milvus")
print("Connected to Milvus")

Connected to Milvus


In [187]:
data_base_name="jio2"
collection_name="jio_cloud2"

In [188]:
if data_base_name not in milvus_client.list_databases():
    milvus_client.create_database(data_base_name)

In [192]:
milvus_client.list_collections()

['jio_cloud', 'jio_cloud2']

In [190]:
fields=[
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=25000),
    FieldSchema(name="keyword_text", dtype=DataType.ARRAY, element_type=DataType.VARCHAR,max_capacity=750,max_length=712)
]
schema = CollectionSchema(fields, description="RAG KB Collection for jio cloud", enable_dynamic_field=True)

In [191]:
if collection_name not in milvus_client.list_collections():
    milvus_client.create_collection(
        collection_name=collection_name,
        schema=schema
    )

In [175]:
if milvus_client:
    print(True)


True


In [74]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")


In [75]:
#tokenization for keyword
tokenized_corpus = [chunk.split() for chunk in chunked_text]

In [76]:
#Embeddings for vector
embeddings = embedding_model.encode(chunked_text).tolist()

In [98]:
# Data formating to store the data in milvus collection
data = [
    {
        "embedding": embeddings[i],
        "text": chunked_text[i],
        "keyword_text":tokenized_corpus[i]
    }
    for i in range(len(chunked_text))
]

In [100]:
result = milvus_client.insert(
    collection_name=collection_name,
    data=data
)

In [196]:
type(result['insert_count'])

int

In [102]:
index_params = milvus_client.prepare_index_params()
index_params.add_index(
    field_name="embedding",
    index_type="HNSW",
    metric_type="COSINE",
    params={
        "M": 16,
        "efConstruction": 200
    }
)

In [ ]:
milvus_client.create_index(
    collection_name=collection_name,
    index_params=index_params
)

milvus_client.load_collection(collection_name=collection_name)

In [125]:
from typing import List, Tuple
import json

from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder
from pymilvus import connections, Collection, utility

In [ ]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [127]:
# === Helper: paginate and fetch all docs ===
def fetch_all_docs(collection: Collection, batch_size: int = 16384):
    docs = []
    offset = 0

    while True:
        chunk = collection.query(
            expr="",                   # no filter
            output_fields=["text", "keyword_text"],
            limit=batch_size,
            offset=offset
        )
        if not chunk:
            break
        docs.extend(chunk)
        offset += len(chunk)
        if len(chunk) < batch_size:
            break

    texts, tokens = [], []
    for row in docs:
        texts.append(row["text"])
        kt = row["keyword_text"]
        if isinstance(kt, str):
            try:
                kt = json.loads(kt)
            except json.JSONDecodeError:
                kt = kt.split()
        tokens.append(kt)
    return texts, tokens

In [128]:
# === Keyword search ===
def keyword_search(query: str, texts: List[str], token_corpus: List[List[str]], top_k: int = 5):
    bm25 = BM25Okapi(token_corpus)
    q_tokens = query.split()
    scores = bm25.get_scores(q_tokens)
    ranked = sorted(zip(texts, scores), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

In [129]:
# === Vector search ===
def vector_search(collection: Collection, query: str, top_k: int = 5, ef: int = 200):
    q_emb = embed_model.encode([query]).tolist()[0]
    results = collection.search(
        data=[q_emb],
        anns_field="embedding",
        param={"metric_type": "COSINE", "params": {"ef": ef}},
        limit=top_k,
        output_fields=["text"]
    )
    hits = results[0]
    return [(hit.entity.get("text"), hit.score) for hit in hits if hit.entity is not None]

In [131]:
# === Hybrid retrieval ===
def hybrid_retrieve(collection_name: str, query: str, top_k: int = 5):
    # Load or confirm collection
    if not utility.has_collection(collection_name):
        raise ValueError(f"Collection '{collection_name}' does not exist")

    collection = Collection(name=collection_name)
    collection.load()

    # Fetch all docs
    texts, tokens = fetch_all_docs(collection)

    # BM25 and vector
    kw = keyword_search(query, texts, tokens, top_k)
    vec = vector_search(collection, query, top_k)

    # Merge & dedupe
    seen = set()
    candidates = []
    for txt, _ in kw + vec:
        if txt not in seen:
            seen.add(txt)
            candidates.append(txt)

    # Rerank with cross-encoder
    pairs = [(query, txt) for txt in candidates]
    ce_scores = cross_encoder.predict(pairs)
    reranked = sorted(zip(candidates, ce_scores), key=lambda x: x[1], reverse=True)

    return reranked[:top_k]


In [ ]:
# === Example usage ===
if __name__ == "__main__":
    connections.connect(alias="default", host="127.0.0.1", port="19530")
    coll_name = "jio_cloud"
    query_str = "What services are available?"
    results = hybrid_retrieve(coll_name, query_str, top_k=5)

    print(f"Top results for '{query_str}':\n")
    for i, (txt, score) in enumerate(results, 1):
        print(f"{i}. [{score:.3f}] {txt[:200]}...\n")


In [133]:
results

[('Integrate a service point picker in your checkout, access a database of all available service point locations for carriers across the globe, and let your customers choose a convenient place to have their parcel delivered.\n- \nShipping Products API\n\nCompare rates, service levels and extra value services across 50+ carriers on both a national and international scale.\n- \nReturn Portal API',
  np.float32(-2.864584)),
 ("Plug and play integrations\n\nIf you use an e-commerce system such as Shopify, WooCommerce, Lightspeed and Magento, you don't need any development skills to connect your store. We provide plug-and-play apps and plugins for 80+ of the most popular e-commerce platforms, available to download from the marketplace or app-store of your particular system.\n\nYou can find more information about integrating your store, marketplace, ERP or WMS system in the \nintegrations section\n of our Help Center.\n\nSendcloud API",
  np.float32(-5.4079375)),
 ('|     |     |\n| --- | --

## Method 2

In [139]:
from typing import List, Tuple
import json

from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder
from pymilvus import utility, Collection
from pymilvus import MilvusClient

# --- Connect ---
milvus_client = MilvusClient(uri="http://127.0.0.1:19530", token="root:Milvus")


In [140]:
def get_collection_handle(name: str) -> Collection:
    if not milvus_client.has_collection(name):
        raise ValueError(f"No collection named '{name}'")
    return Collection(name=name)

# --- Models ---
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [152]:
# --- Pagination fetch ---
def fetch_all_docs(collection: Collection, batch_size: int = 500) -> Tuple[List[str], List[List[str]]]:
    docs = []
    offset = 0
    while True:
        rows = milvus_client.query(
            collection_name=collection.name,
            filter="",
            output_fields=["text", "keyword_text"],
            limit=batch_size,
            offset=offset
        )
        if not rows:
            break
        docs.extend(rows)
        offset += len(rows)
        if len(rows) < batch_size:
            break

    texts, tokens = [], []
    for row in docs:
        texts.append(row["text"])
        kt = row["keyword_text"]
        if isinstance(kt, str):
            try:
                kt = json.loads(kt)
            except:
                kt = kt.split()
        tokens.append(kt)
    return texts, tokens

In [142]:
# --- BM25 keyword search ---
def keyword_search(query: str, texts: List[str], token_corpus: List[List[str]], top_k: int = 5):
    bm25 = BM25Okapi(token_corpus)
    scores = bm25.get_scores(query.split())
    ranked = sorted(zip(texts, scores), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

In [165]:
# --- Vector search with correct API usage ---
def vector_search(collection: Collection, query: str, top_k: int = 5, ef: int = 200):
    q_emb = embed_model.encode([query]).tolist()[0]
    results = milvus_client.search(
        collection_name=collection.name,
        anns_field="embedding",
        data=[q_emb],
        limit=top_k,
        search_params={"metric_type": "COSINE", "params": {"ef": ef}},
        output_fields=["text"]
    )
    # results is List[List[dict]]
    candidates = []
    for hits in results:
        for hit in hits:
            candidates.append((hit["entity"]["text"], hit["distance"]))
    return candidates

In [144]:
# --- Hybrid retrieval with reranking ---
def hybrid_retrieve(collection_name: str, query: str, top_k: int = 5):
    coll = get_collection_handle(collection_name)
    coll.load()
    texts, tokens = fetch_all_docs(coll)

    kw = keyword_search(query, texts, tokens, top_k)
    vec = vector_search(coll, query, top_k)

    seen = set(); candidates = []
    for txt, _ in kw + vec:
        if txt not in seen:
            seen.add(txt)
            candidates.append(txt)

    pairs = [(query, txt) for txt in candidates]
    ce_scores = cross_encoder.predict(pairs)

    reranked = sorted(zip(candidates, ce_scores), key=lambda x: x[1], reverse=True)
    return reranked[:top_k]

In [167]:

collection_name = "jio_cloud"
query_str = "What is jio cloud"
results = hybrid_retrieve(collection_name, query_str, top_k=5)

In [205]:
milvus_client.list_databases()

['jio2', 'jio_new', 'jiuww', 'default', 'jio']

In [198]:
def get_milvus_client(
    uri: str = "http://127.0.0.1:19530",
    token: str = "root:Milvus",
    db_name: str = None) -> MilvusClient:
    """
    Returns a MilvusClient instance.
    
    If db_name is provided, connects to that database.
    If not, connects using default database.
    """
    if db_name:
        return MilvusClient(uri=uri, token=token, db_name=db_name)
    else:
        return MilvusClient(uri=uri, token=token)

In [199]:
milvus_client_1=get_milvus_client(db_name='jio_new')

In [203]:
def create_database(database_name):
    milvus_client = get_milvus_client()
    milvus_client.create_database(database_name)
    if database_name in milvus_client.list_databases():
        return True
    else:
        return False

In [204]:
create_database("jiuww")

True